In [66]:
##  import libraries

import pandas as pd
import numpy as np
from yahooquery import Ticker
import yfinance as yf
import datetime as dt

In [67]:
##  get tickers from file 

raw_data = pd.read_excel('tickers.xlsx') #  <-- loading file with tickers
# raw_data = pd.read_excel('All Public comps with tickers.xlsx') #  <-- loading file with tickers
tickers = raw_data['Public - Ticker (Yahoo)'][:100] #  <-- getting tickers column from file 

In [68]:
##  needed information

summary_detail_keys = ['previousClose', 'marketCap', 'currency']
financial_data_keys = ['totalRevenue', 'totalCash', 'totalDebt', 'ebitda', 'freeCashflow', 'operatingCashflow',
                  'grossProfits', 'revenueGrowth']
key_stats_keys = ['enterpriseValue', 'sharesOutstanding', 'floatShares', 'lastFiscalYearEnd', 'profitMargins']

keys = summary_detail_keys + financial_data_keys + key_stats_keys + ['growthFromFeb']

In [69]:
## parsing
    
def to_dict(x):
    if isinstance(x, dict):
        return x
    else:
        return dict()
    
    
def parse():
    counter = 0
    empty_row = pd.DataFrame({ key: [np.nan] for key in keys })
    result = pd.DataFrame()
    prices = []
    for ticker in tickers:
        
        if ticker is np.nan:
            result = result.append(empty_row, ignore_index = True)
            continue
            
        try:
            ticker_info = Ticker(ticker)
        except TypeError:
            result = result.append(empty_row, ignore_index = True)
            continue
            
        summary_detail = to_dict(ticker_info.summary_detail.get(ticker))
        financial_data = to_dict(ticker_info.financial_data.get(ticker))
        key_stats = to_dict(ticker_info.key_stats.get(ticker))
                    
        row = dict()
        row.update({ key: summary_detail.get(key) for key in summary_detail_keys })
        row.update({ key: financial_data.get(key) for key in financial_data_keys })
        row.update({ key: key_stats.get(key) for key in key_stats_keys })
            
        try:
            price = yf.download(ticker, start=dt.date(2020,2,4), end=dt.date(2020,2,7)).Close[0]
            row['growthFromFeb'] = row['previousClose'] / price - 1
        except:
            row['growthFromFeb'] = np.nan
            
        row_df =  pd.DataFrame(row, index=[0])
        result = result.append(row_df)
        counter += 1
        if counter % 100 == 0:
            print(f'{counter} tickers downloaded')
    result.reset_index(drop=True, inplace=True)
    return result
     
parsed_df = parse()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [71]:
def to_dollar():
    not_dividing = ['previousClose', 'currency', 'lastFiscalYearEnd', 'revenueGrowth', 'profitMargins', 'growthFromFeb']
    
    def remove_cents(x):
        try:
            return float(x) / 100
        except TypeError:
            return x
        
    currencies = ['GBp', 'ZAc', 'ILA']
    
    for currency in currencies:
        currency_data = parsed_df[parsed_df['currency'] == currency]
        for column in currency_data.columns:
            if column not in not_dividing:
                currency_data[column] = currency_data[column].apply(remove_cents)
            else:
                currency_data[column] = currency_data[column]
        parsed_df[parsed_df['currency'] == currency] = currency_data
        
to_dollar()

C:\Users\www-w\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\www-w\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [72]:
parsed_df

,previousClose,marketCap,currency,totalRevenue,totalCash,totalDebt,ebitda,freeCashflow,operatingCashflow,grossProfits,revenueGrowth,enterpriseValue,sharesOutstanding,floatShares,lastFiscalYearEnd,profitMargins,growthFromFeb
0,7.440000,293916896,USD,1.459418e+09,1.225740e+08,2.815700e+07,-1.039520e+08,-6.215362e+07,-8.161200e+07,2.930930e+08,-0.180,268377904,40373200,38025207,2019-12-31,-0.08349,-5.822786e-02
1,33.550000,27738310656,USD,NaN,NaN,NaN,NaN,NaN,NaN,2.023680e+11,0.008,27152809984,809324032,138248672,2019-12-31,0.0546,-1.428206e-01
2,75.000000,294607,GBp,1.139600e+05,1.110100e+05,2.379000e+04,-3.001000e+04,-3.450875e+04,-1.910000e+04,8.990000e+04,0.306,207324,387641,262687,2019-12-31,-0.25842,-1.935484e-01
3,0.007000,7051,USD,2.377600e+04,1.483000e+03,2.354698e+06,-1.012831e+06,2.243240e+05,-6.920550e+05,2.377600e+04,NaN,2358829,1007300,74689,2018-12-31,0,-8.963055e-01
4,0.075000,2612820,USD,NaN,4.328500e+05,0.000000e+00,NaN,2.134890e+05,-3.377560e+05,NaN,NaN,2179972,34837600,14695207,2019-12-31,0,-1.815801e-01
5,2.600000,482360416,INR,1.490740e+10,4.562600e+09,7.521000e+10,-1.557600e+09,NaN,NaN,5.046100e+09,-0.294,71077191680,165891008,142374216,2019-03-31,-2.12928,-3.333333e-01
6,29.320000,679376320,USD,1.940008e+09,3.144360e+08,2.244750e+08,1.658720e+08,-2.614789e+08,1.663590e+08,1.056356e+09,0.191,610786304,23709000,23073920,2019-12-31,0.04052,-3.084906e-01
7,1.800000,42865240,INR,9.416200e+07,9.605000e+06,5.110700e+07,-4.489300e+07,NaN,NaN,-1.514700e+07,3.095,66616284,14283100,6389771,2019-03-31,-0.07813,-6.288660e-01
8,0.527753,9166753,USD,2.365900e+07,6.507606e+06,1.783349e+06,-2.456429e+06,-1.681640e+05,-1.227772e+06,1.303801e+07,-0.333,4361258,17215500,15022735,2019-12-31,-0.15828,-4.634748e-01
9,1.770000,156449200,USD,2.855830e+08,2.393900e+07,2.496200e+07,1.788500e+07,-2.667875e+07,-1.293100e+07,9.684200e+07,-0.169,144131584,80852304,57908041,2019-12-31,-0.00821,-1.237624e-01


In [70]:
parsed_df

,previousClose,marketCap,currency,totalRevenue,totalCash,totalDebt,ebitda,freeCashflow,operatingCashflow,grossProfits,revenueGrowth,enterpriseValue,sharesOutstanding,floatShares,lastFiscalYearEnd,profitMargins,growthFromFeb
0,7.440000,293916896,USD,1.459418e+09,1.225740e+08,2.815700e+07,-1.039520e+08,-6.215362e+07,-8.161200e+07,2.930930e+08,-0.180,268377904,40373200,38025207,2019-12-31,-0.08349,-5.822786e-02
1,33.550000,27738310656,USD,NaN,NaN,NaN,NaN,NaN,NaN,2.023680e+11,0.008,27152809984,809324032,138248672,2019-12-31,0.0546,-1.428206e-01
2,75.000000,29460716,GBp,1.139600e+07,1.110100e+07,2.379000e+06,-3.001000e+06,-3.450875e+06,-1.910000e+06,8.990000e+06,0.306,20732438,38764100,26268708,2019-12-31,-0.25842,-1.935484e-01
3,0.007000,7051,USD,2.377600e+04,1.483000e+03,2.354698e+06,-1.012831e+06,2.243240e+05,-6.920550e+05,2.377600e+04,NaN,2358829,1007300,74689,2018-12-31,0,-8.963055e-01
4,0.075000,2612820,USD,NaN,4.328500e+05,0.000000e+00,NaN,2.134890e+05,-3.377560e+05,NaN,NaN,2179972,34837600,14695207,2019-12-31,0,-1.815801e-01
5,2.600000,482360416,INR,1.490740e+10,4.562600e+09,7.521000e+10,-1.557600e+09,NaN,NaN,5.046100e+09,-0.294,71077191680,165891008,142374216,2019-03-31,-2.12928,-3.333333e-01
6,29.320000,679376320,USD,1.940008e+09,3.144360e+08,2.244750e+08,1.658720e+08,-2.614789e+08,1.663590e+08,1.056356e+09,0.191,610786304,23709000,23073920,2019-12-31,0.04052,-3.084906e-01
7,1.800000,42865240,INR,9.416200e+07,9.605000e+06,5.110700e+07,-4.489300e+07,NaN,NaN,-1.514700e+07,3.095,66616284,14283100,6389771,2019-03-31,-0.07813,-6.288660e-01
8,0.527753,9166753,USD,2.365900e+07,6.507606e+06,1.783349e+06,-2.456429e+06,-1.681640e+05,-1.227772e+06,1.303801e+07,-0.333,4361258,17215500,15022735,2019-12-31,-0.15828,-4.634748e-01
9,1.770000,156449200,USD,2.855830e+08,2.393900e+07,2.496200e+07,1.788500e+07,-2.667875e+07,-1.293100e+07,9.684200e+07,-0.169,144131584,80852304,57908041,2019-12-31,-0.00821,-1.237624e-01


In [ ]:
## adding data to file

def devide(x):
    try:
        return float(x) / 1000000
    except TypeError:
        return x
    
not_dividing = ['previousClose', 'currency', 'lastFiscalYearEnd', 'revenueGrowth', 'profitMargins', 'growthFromFeb']
for column in parsed_df.columns:
    if column not in not_dividing:
        raw_data[column] = parsed_df[column].apply(devide)
    else:
        raw_data[column] = parsed_df[column]

In [ ]:
## Save result

raw_data.to_excel('Comps_statistics.xlsx')

In [ ]:
# currencies = ['GBp', 'ZAc', 'ILA']
# parsed_df[parsed_df['currency'] == 'ILA']